In [ ]:
import os
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import csv
import torch

documents = []

with open('/bigwork/nhwpshaa/BigDataset/firstOrderEffect.csv', 'r') as file: #load and read the file containing causal relationships
    csvfile = csv.DictReader(file)
    for row in csvfile:
        content= f"Cause:{row['cause']} Effect:{row['effect']}" #form a sentence to provide as a content
        documents.append(Document(page_content= content)) #create a document object and append all of it to the array
        
with open('/bigwork/nhwpshaa/BigDataset/cause.csv', 'r') as file: #load and read the file containing causal relationships
    csvfile = csv.DictReader(file)
    for row in csvfile:
        content= f"Cause:{row['cause']} Effect:{row['effect']}" #form a sentence to provide as a content
        documents.append(Document(page_content= content)) #load and read the file containing causal relationships
        

model = HuggingFaceEmbeddings(model_name='/bigwork/nhwpshaa/all-MiniLM-L6-v2') #initialize embeddings
vectorDB = FAISS.from_documents(documents, model) #initialize vector database
modelName = "/bigwork/nhwpshaa/alpaca-native"
tokenizer = LlamaTokenizer.from_pretrained(modelName)
llm = LlamaForCausalLM.from_pretrained(modelName, top_k = 50) #set the same settings as for other models
pipelineModel = pipeline("text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    min_length=500,
    max_length=900,
    do_sample=True,
    top_k=30,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
    )
pipe = HuggingFacePipeline(pipeline=pipelineModel) 
prompt = PromptTemplate(
    template="{context}\nAct as an expert debater to generate an essay consisting of around 400 words for the following input topic. Assume only the 'con' stance. Segment the essay into well-structured pararaphs \nTopic: {question}\nAnswer:",
    input_variables=["context", "question"]
)

chain = RetrievalQA.from_llm(
    llm=pipe,
    retriever=vectorDB.as_retriever(), #initialize the vector databse as a retriever
    prompt=prompt
)


question = "Crime"
response = chain({"query": question})
print(f"Question: {question}\nAnswer: {response}")


/home/nhwpshaa/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/nhwpshaa/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/home/nhwpshaa/.local/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/nhwpshaa/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
 